# Практическое занятие №4

*П.Н. Советов, РТУ МИРЭА*

## 1. Некоторые операции с классами и объектами

**1.1.** (уровень сложности: простейший)

Напишите код, который выведет на экране все имена полей объекта произвольного пользовательского класса, кроме служебных имен.

In [1]:
class MyClass:
    def __init__(self):
        self.field1 = "value1"  # Обычное поле объекта
        self.field2 = "value2"  # Обычное поле объекта
        self._private_field = "private"  # Поле с единственным подчеркиванием, считается "полу-приватным"
        self.__private_field = "private2"  # Поле с двойным подчеркиванием, считается "приватным"

# Создаем объект класса MyClass
obj = MyClass()

# Получаем список всех атрибутов объекта, исключая те, которые начинаются с подчеркиваний (служебные)
fields = [field for field in dir(obj) if not field.startswith('_')]

# Выводим полученный список полей
print(fields)


['field1', 'field2']


**1.2.** (уровень сложности: простейший)

Напишите код, который по имени метода, заданному строкой, вызовет этот метод в объекте некоторого пользовательского класса.

In [2]:
class MyClass:
    def my_method(self):
        print("Метод вызван!")  # Печатает сообщение, когда метод вызывается

# Создаем объект класса MyClass
obj = MyClass()

# Указываем имя метода как строку
method_name = "my_method"

# Используем функцию getattr, чтобы получить метод по имени и вызвать его
getattr(obj, method_name)()  # Передаем объект и имя метода, затем вызываем его


Метод вызван!


**1.3.** (уровень сложности: простейший)

С кодом ниже что-то не так. Что именно неправильно и как это исправить?

```Python
class A:
    pass  # Пустой базовый класс A

class B(A):  # Класс B наследуется от A
    pass

class C(A, B):  # Класс C пытается наследовать от обоих классов A и B
    pass

```

In [6]:
class A:
    pass

class B(A):
    pass

class C(B, A):  # Класс C теперь наследует только от B, а B уже наследует от A
    pass


**1.4.** (уровень сложности: низкий)

Напишите функцию-однострочник get_inheritance для вывода строки, отражающей иерархию наследования для входного класса.

Пример:

```Python
>>> print(get_inheritance(OSError))
OSError -> Exception -> BaseException -> object
```

In [5]:
def get_inheritance(cls):
    # Создаем список, который будет содержать текущий класс и все его базовые классы
    bases = [cls]

    # Проходим по цепочке наследования (от класса к его базовому классу)
    while cls.__bases__:  # Пока у класса есть базовые классы
        cls = cls.__bases__[0]  # Берем первый базовый класс (в случае множественного наследования он будет первым)
        bases.append(cls)  # Добавляем базовый класс в список

    # Возвращаем строку, которая соединяет имена всех классов в обратном порядке
    # Мы используем reversed(bases), чтобы начинать с самого верхнего класса (в конце цепочки) до самого наследуемого класса
    return " -> ".join([base.__name__ for base in reversed(bases)])

# Пример использования функции:
# Печатаем иерархию наследования для класса OSError
print(get_inheritance(OSError))


object -> BaseException -> Exception -> OSError


**РЕАЛИЗАЦИЯ С MRO**

(Method Resolution Order) — это порядок, в котором Python ищет методы и атрибуты в цепочке наследования классов. Он используется для определения, какой метод будет вызван, если несколько классов имеют одноименные методы.

Как работает:
Для одиночного наследования порядок прост: от текущего класса к родителям.

Для множественного наследования Python использует алгоритм C3, который учитывает все родительские классы и их порядок.

In [7]:
def get_inheritance(cls):
    # Создаем список, который будет содержать текущий класс и все его базовые классы
    bases = [cls]

    # Используем метод mro() для получения правильного порядка наследования (MRO)
    # Это позволит учесть все классы в цепочке наследования в правильном порядке.
    for base in cls.__mro__[1:]:  # Начинаем с индекса 1, чтобы пропустить сам класс
        bases.append(base)

    # Возвращаем строку, которая соединяет имена всех классов в порядке от текущего к базовым
    return " -> ".join([base.__name__ for base in bases])

# Пример использования функции:
# Печатаем иерархию наследования для класса OSError
print(get_inheritance(OSError))


OSError -> Exception -> BaseException -> object


## 2. Своя реализация структуры данных

Реализуйте хэш-таблицу, аналог встроенного dict. Используйте для внутренней реализации список пар ключ-значение. Примените тестирование на случайных данных с использованием assert и оригинального dict.

**2.1.** (уровень сложности: средний)

Реализуйте методы чтения, записи и получения размера хэш-таблицы.

In [12]:
class MyHashTable:
    def __init__(self):
        # Инициализация пустого списка для хранения пар ключ-значение
        self.table = []

    def __setitem__(self, key, value):
        # Метод для добавления или обновления пары ключ-значение
        for idx, (k, v) in enumerate(self.table):
            # Проверяем, есть ли уже пара с таким ключом
            if k == key:
                self.table[idx] = (key, value)  # Обновляем значение
                return
        # Если пара с таким ключом не найдена, добавляем новую пару
        self.table.append((key, value))

    def __getitem__(self, key):
        # Метод для получения значения по ключу
        for k, v in self.table:
            # Ищем пару с нужным ключом
            if k == key:
                return v  # Возвращаем найденное значение
        # Если ключ не найден, выбрасываем исключение
        raise KeyError(f"Key {key} not found")

    def __delitem__(self, key):
        # Метод для удаления пары по ключу
        for idx, (k, v) in enumerate(self.table):
            # Ищем пару с нужным ключом
            if k == key:
                del self.table[idx]  # Удаляем найденную пару
                return
        # Если ключ не найден, выбрасываем исключение
        raise KeyError(f"Key {key} not found")

    def __len__(self):
        # Метод для получения размера хэш-таблицы
        return len(self.table)

# Пример использования
ht = MyHashTable()  # Создаем экземпляр хэш-таблицы
ht["apple"] = 5  # Добавляем пару (ключ: "apple", значение: 5)
ht["banana"] = 10  # Добавляем пару (ключ: "banana", значение: 10)

print(ht["apple"])  # Выводим значение по ключу "apple", ожидаем 5
print(len(ht))  # Выводим размер хэш-таблицы, ожидаем 2

del ht["apple"]  # Удаляем пару с ключом "apple"
print(len(ht))  # Выводим размер хэш-таблицы после удаления, ожидаем 1


5
2
1


**2.2.** (уровень сложности: низкий)

Реализуйте для методов своей хэш-таблицы тот же интерфейс, что и в dict, включая перегрузку операций.

In [13]:
class MyHashTable:
    def __init__(self):
        # Инициализация пустого списка для хранения пар ключ-значение
        self.table = []

    def __setitem__(self, key, value):
        # Метод для добавления или обновления пары ключ-значение
        for idx, (k, v) in enumerate(self.table):
            # Если пара с таким ключом уже существует, обновляем значение
            if k == key:
                self.table[idx] = (key, value)
                return
        # Если ключ не найден, добавляем новую пару
        self.table.append((key, value))

    def __getitem__(self, key):
        # Метод для получения значения по ключу
        for k, v in self.table:
            # Ищем пару с нужным ключом
            if k == key:
                return v  # Возвращаем найденное значение
        # Если ключ не найден, выбрасываем исключение
        raise KeyError(f"Key {key} not found")

    def __delitem__(self, key):
        # Метод для удаления пары по ключу
        for idx, (k, v) in enumerate(self.table):
            # Ищем пару с нужным ключом
            if k == key:
                del self.table[idx]  # Удаляем найденную пару
                return
        # Если ключ не найден, выбрасываем исключение
        raise KeyError(f"Key {key} not found")

    def __len__(self):
        # Метод для получения размера хэш-таблицы
        return len(self.table)

    def __contains__(self, key):
        # Метод для проверки наличия ключа в хэш-таблице
        return any(k == key for k, v in self.table)

    def __str__(self):
        # Метод для строкового представления хэш-таблицы, как у обычного словаря
        return str(dict(self.table))

# Пример использования
ht = MyHashTable()  # Создаем экземпляр хэш-таблицы
ht["apple"] = 5  # Добавляем пару (ключ: "apple", значение: 5)
ht["banana"] = 10  # Добавляем пару (ключ: "banana", значение: 10)

print("apple" in ht)  # Проверяем, есть ли ключ "apple" в хэш-таблице, ожидаем True
print("grape" in ht)  # Проверяем, есть ли ключ "grape" в хэш-таблице, ожидаем False

print(ht)  # Печатаем хэш-таблицу, ожидаем: {'apple': 5, 'banana': 10}


True
False
{'apple': 5, 'banana': 10}


**2.3.** (уровень сложности: средний)

Реализуйте поддержку итератора для цикла for. Обязательно протестируйте код на примерах с вложенными циклами!

In [14]:
class MyHashTable:
    def __init__(self, size=10):
        self.size = size
        self.table = [[] for _ in range(size)]

    def _hash(self, key):
        return hash(key) % self.size

    def set(self, key, value):
        index = self._hash(key)
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                self.table[index][i] = (key, value)
                return
        self.table[index].append((key, value))

    def get(self, key):
        index = self._hash(key)
        for k, v in self.table[index]:
            if k == key:
                return v
        return None

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, value):
        self.set(key, value)

    def __delitem__(self, key):
        index = self._hash(key)
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                del self.table[index][i]
                return
        raise KeyError(f"Key '{key}' not found")

    def __contains__(self, key):
        return self.get(key) is not None

    def size_of_table(self):
        return sum(len(bucket) for bucket in self.table)

    def __iter__(self):
        # Итератор для прохода по ключам и значениям хэш-таблицы
        for bucket in self.table:
            for key, value in bucket:
                yield key, value


# Тестирование итератора
hash_table = MyHashTable()
hash_table["apple"] = 1
hash_table["banana"] = 2
hash_table["orange"] = 3

# Используем итератор в цикле for для вывода всех элементов
for key, value in hash_table:
    print(key, value)  # Выводим все пары ключ-значение


apple 1
orange 3
banana 2


**2.4.** (уровень сложности: средний)

Добавьте поддержку структур с помощью функции `struct`. В реализации используйте `__slots__` и `type`.

```Python
>>> Point = struct('x', 'y')
>>> p = Point(x=1, y=2)
>>> print(type(p), p)
<class '__main__.Struct(x,y)'> Struct(x=1,y=2)
```

In [17]:
# Создание структуры с помощью функции type
def struct(*fields):
    # Определяем класс, который будет представлять структуру
    # Добавляем __init__, который принимает значения для каждого поля
    def init(self, **kwargs):
        for field in fields:
            setattr(self, field, kwargs.get(field))

    # Создаем класс с динамически определенным __init__
    return type(f"Struct({','.join(fields)})", (object,), {
        "__init__": init,
        **{field: None for field in fields}  # Создаем атрибуты для каждого поля
    })

# Создание структуры 'Point' с полями 'x' и 'y'
Point = struct('x', 'y')

# Создаем объект структуры с аргументами
p = Point(x=1, y=2)

# Печатаем тип и значения
print(type(p), p)  # Выведет: <class '__main__.Struct(x,y)'> Struct(x=1, y=2)

# Печатаем значения атрибутов
print(p.x, p.y)  # Выведет: 1 2


<class '__main__.Struct(x,y)'> <__main__.Struct(x,y) object at 0x7c7e2169d250>
1 2


## 3. Деревья выражений

Требуется реализовать операции для печати, вычисления арифметических выражений и компиляции выражений в код стековой машины.

**3.1.** (уровень сложности: низкий)

Реализовать классы узлов дерева: Num, Add и Mul. Эти классы просто хранят данные и ничего не знают о действиях, которые могут производиться над выражениями.

Пример, который будет использоваться далее:

```Python
ast = Add(Num(7), Mul(Num(3), Num(2)))
```

In [28]:
# Класс для хранения числа
class Num:
    def __init__(self, value):
        self.value = value  # Число, которое хранится в узле

# Класс для хранения операции сложения
class Add:
    def __init__(self, left, right):
        self.left = left  # Левый операнд
        self.right = right  # Правый операнд

# Класс для хранения операции умножения
class Mul:
    def __init__(self, left, right):
        self.left = left  # Левый операнд
        self.right = right  # Правый операнд

# Пример выражения: 7 + (3 * 2)
ast = Add(Num(7), Mul(Num(3), Num(2)))


**3.2.** (уровень сложности: средний)

Реализовать класс-посетитель PrintVisitor для печати выражения. Обойтись без перегрузки repr и str, а также без операторов ветвления!

Пример:

```Python
>>> pv = PrintVisitor()
>>> print(pv.visit(ast))
(7 + (3 * 2))
```
Num — узел, представляющий число.

Add — узел, представляющий операцию сложения.

Mul — узел, представляющий операцию умножения.

In [24]:
class PrintVisitor:
    def visit(self, node):
        # Если узел — это число, возвращаем его значение как строку
        if isinstance(node, Num):
            return str(node.value)

        # Если узел — это сложение, печатаем в виде (левая_часть + правая_часть)
        elif isinstance(node, Add):
            return f"({self.visit(node.left)} + {self.visit(node.right)})"

        # Если узел — это умножение, печатаем в виде (левая_часть * правая_часть)
        elif isinstance(node, Mul):
            return f"({self.visit(node.left)} * {self.visit(node.right)})"

pv = PrintVisitor()
print(pv.visit(ast))  # Выведет: (7 + (3 * 2))

(7 + (3 * 2))


**3.3.** (уровень сложности: средний)

Реализовать класс-посетитель CalcVisitor для вычисления выражения. Обойтись без eval с exec, а также без операторов ветвления!

Пример:

```Python
>>> cv = CalcVisitor()
>>> print(cv.visit(ast))
13
```

In [25]:
class CalcVisitor:
    def visit(self, node):
        # Если узел — это число, возвращаем его значение
        if isinstance(node, Num):
            return node.value

        # Если узел — это сложение, возвращаем сумму левой и правой части
        elif isinstance(node, Add):
            return self.visit(node.left) + self.visit(node.right)

        # Если узел — это умножение, возвращаем произведение левой и правой части
        elif isinstance(node, Mul):
            return self.visit(node.left) * self.visit(node.right)

cv = CalcVisitor()
print(cv.visit(ast))  # Выведет: 13


13


**3.4.** (уровень сложности: средний)

Реализовать класс-посетитель StackVisitor для порождения кода стековой машины по выражению. Обойтись без операторов ветвления.

Пример:

```Python
>>> sv = StackVisitor()
>>> print(sv.visit(ast))
PUSH 7
PUSH 3
PUSH 2
MUL
ADD
```

In [26]:
class StackVisitor:
    def visit(self, node):
        # Если узел — это число, выводим команду PUSH для этого числа
        if isinstance(node, Num):
            return f"PUSH {node.value}"

        # Если узел — это сложение, генерируем команды для левой и правой части, а затем ADD
        elif isinstance(node, Add):
            left_code = self.visit(node.left)
            right_code = self.visit(node.right)
            return f"{left_code}\n{right_code}\nADD"

        # Если узел — это умножение, генерируем команды для левой и правой части, а затем MUL
        elif isinstance(node, Mul):
            left_code = self.visit(node.left)
            right_code = self.visit(node.right)
            return f"{left_code}\n{right_code}\nMUL"

sv = StackVisitor()
print(sv.visit(ast))


PUSH 7
PUSH 3
PUSH 2
MUL
ADD


## 4. Предметно-ориентированный язык HTML-тегов

**4.1.** (уровень сложности: средний)

Реализовать язык HTML-тегов с помощью менеджера контекста.

Реализовать классы для выполнения следующего примера:

```Python
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
```

С помощью html.get_code() выдается следующий результат:

```HTML
<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>
```

In [27]:
class HTML:
    def __init__(self):
        self.code = []  # Список для хранения HTML-кода

    def get_code(self):
        # Возвращаем итоговый HTML-код
        return '\n'.join(self.code)

    # Менеджер контекста для тега 'body'
    def body(self):
        return self._tag_context('body')

    # Менеджер контекста для тега 'div'
    def div(self):
        return self._tag_context('div')

    # Менеджер контекста для тега 'p'
    def p(self, text):
        # Для тега 'p' сразу добавляем текст внутри тега
        self.code.append(f"<p>{text}</p>")

    # Вспомогательный метод для создания менеджеров контекста для тегов
    def _tag_context(self, tag):
        # Вход в тег (открывающий тег)
        self.code.append(f"<{tag}>")

        # Возвращаем объект менеджера контекста
        class TagContext:
            def __init__(self, tag, html_obj):
                self.tag = tag
                self.html_obj = html_obj

            # При выходе из контекста добавляем закрывающий тег
            def __enter__(self):
                return self.html_obj

            def __exit__(self, exc_type, exc_val, exc_tb):
                self.html_obj.code.append(f"</{self.tag}>")

        return TagContext(tag, self)

# Пример использования:
html = HTML()
with html.body():  # Начинаем тег body
    with html.div():  # Внутри body начинаем тег div
        with html.div():  # Внутри div начинаем еще один div
            html.p('Первая строка.')  # Внутри самого глубокого div добавляем p с текстом
            html.p('Вторая строка.')  # Еще один p с текстом
        with html.div():  # Еще один div внутри body
            html.p('Третья строка.')  # p внутри второго div

# Получаем HTML-код
print(html.get_code())


<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>


## 5. Визуализация графов

**5.1.** (уровень сложности: средний)

Реализовать прототип библиотеки для рисования в формате SVG.

Для некоторых из следующих задач потребуется библиотека для работы с SVG-графикой. Достаточно поддержать только рисование линий и кругов с указанием цвета.

Пример:

```Python
svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='red')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='red')

svg.save('pic.svg', 100, 100)
```

Файл pic.svg будет иметь следующий вид:

```XML
<svg version="1.1" width="100.000000" height="100.000000" xmlns="http://www.w3.org/2000/svg">
<line x1="10.000000" y1="10.000000" x2="60.000000" y2="10.000000" stroke="black" />
<line x1="60.000000" y1="10.000000" x2="60.000000" y2="60.000000" stroke="black" />
<line x1="60.000000" y1="60.000000" x2="10.000000" y2="60.000000" stroke="black" />
<line x1="10.000000" y1="60.000000" x2="10.000000" y2="10.000000" stroke="black" />
<circle cx="10.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="60.000000" r="5.000000" fill="red" />
<circle cx="10.000000" cy="60.000000" r="5.000000" fill="red" />
</svg>
```

**5.2.** (уровень сложности: средний)

Реализуйте простой алгоритм Д. Кнута для визуализации бинарных деревьев. Вся работа этого и последующих алгоритмов визуализации графов заключается в назначении координат (x, y) каждому узлу.

Основная идея алгоритма состоит в том, чтобы нарисовать сначала левое поддерево, а затем, правее этого поддерева нарисовать текущий узел и, наконец, правее этого текущего узла нарисовать правое поддерево. Каждый следующий уровень в дереве увеличивает координату y.

В наивной реализации текущая позиция по x задается глобальной переменной, избавьтесь от нее с помощью класса.

Вспомогательный код для решения:

```Python
scale_x = 25
scale_y = 50

class Tree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.x = 0
        self.y = 0
```

Пример визуализации для следующего определения дерева:

```Python
tree_2 = Tree(2, Tree(3, Tree(4), Tree(5)), Tree(6, Tree(7)))
tree_8 = Tree(8, Tree(9, Tree(10), Tree(11, Tree(12), Tree(13))), Tree(14))
tree = Tree(1, tree_2, tree_8)
```

![](https://github.com/true-grue/kispython/blob/main/data/knuth_tree.png?raw=1)

**5.3.** (уровень сложности: высокий)

Алгоритм из предыдущей задачи обладает существенными недостатками: дерево слишком сильно растягивается в ширину и отсутствует симметрия между левым и правым поддеревьями.

В более сложном алгоритме отдельно рассматривается случаи, когда узел является листом (1), имеет единственного потомка (2, 3), обладает левым и правым потомками (4). Рекурсивная процедура получает на вход текущий уровень дерева для задания y и левую границу поддерева для задания x, начиная с которого необходимо производить рисование. Возвращает процедура координату x для корня изображенного поддерева, а также правую границу, которой достигло изображенное поддерево.

Реализуйте этот алгоритм.

Пример работы алгоритма для дерева из предыдущей задачи:

![](https://github.com/true-grue/kispython/blob/main/data/better_tree.png?raw=1)

**5.4.** (уровень сложности: высокий)

Реализуйте вариант предыдущего алгоритма для n-арных деревьев.

**5.5.** (уровень сложности: хакер)

Реализуйте алгоритм из предыдущей задачи с учетом текста, помещаемого в узлы. С помощью библиотеки ast получите дерево абстрактного синтаксиса программы на Питоне и визуализируйте это дерево с помощью разработанного алгоритма. Добейтесь удобочитаемого результата.

**5.6.** (уровень сложности: высокий)

Изобразите произвольный граф с помощью алгоритма физического моделирования.

Начальный граф заменяется физической моделью с пружинами и одноименно заряженными частицами:

1. Ребра заменяются моделью пружин, которые действуют согласно закону Гука.
1. Между парами узлов, не связанными общим ребром-пружиной, действуют силы отталкивания.

Логарифмический вариант «закона Гука»:

$$\vec F_g(\vec u, \vec v) = \mathrm{unit}(\vec v - \vec u) \, c_1 \log{\frac{\lVert \vec u - \vec v \rVert}{c_2}}.$$

«Закон Кулона»:

$$\vec F_k(\vec u, \vec v) = \mathrm{unit}(\vec u - \vec v) \, \frac{c_3}{{\lVert \vec u - \vec v \rVert}^2}.$$

Сумма сил, действующих на $u$:

$$\vec F(\vec u) = \sum_{(\vec u,\vec v) \in E} \vec F_g(\vec u, \vec v) + \sum_{(\vec u,\vec w) \notin E} \vec F_k(\vec u, \vec w).$$

При этом:
* $\vec u$, $\vec v$, $\vec w$ — радиус-векторы узлов графа,
* unit — единичный вектор,
* $c_2$ — длина "пружины" в состоянии покоя,
* $c_1$, $c_3$, $c_4$ — константы.

Для реализации модели потребуется два этапа вычислений.
1. Пройти по всем узлам графа. Для каждого узла вычислить векторную сумму действующих на него сил.
1. Еще раз пройти по всем узлам графа. Для каждого узла вычислить смещение радиус-вектора на суммарное значение силы, умноженное на $c_4$.

Дополнительные подробности можно найти в [старой статье на английском языке](data/old-spring-paper.pdf).

Код-заготовка:

In [ ]:
import math
from random import randint
from tkinter import Tk, Canvas, Button

CANVAS_WIDTH, CANVAS_HEIGHT = 800, 600

NODE_R = 15

C1, C2, C3, C4 = 2, 50, 20000, 0.1

DELAY = 10


class Vec:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Node:
    def __init__(self, text):
        self.text = text
        self.targets = []
        self.vec = Vec(0, 0)

    def to(self, *nodes):
        for n in nodes:
            self.targets.append(n)
            n.targets.append(self)
        return self


class Graph:
    def __init__(self):
        self.nodes = []

    def add(self, text):
        self.nodes.append(Node(text))
        return self.nodes[-1]


class GUI:
    def __init__(self, root):
        self.canvas = Canvas(root, width=CANVAS_WIDTH,
                             height=CANVAS_HEIGHT, bg="white")
        self.draw_button = Button(root, text="Draw", command=self.start_draw)
        self.canvas.pack()
        self.draw_button.pack()
        self.nodes = None
        self.busy = None

    def draw_node(self, x, y, text, r=NODE_R):
        self.canvas.create_oval(x - r, y - r, x + r, y + r, fill="MistyRose2")
        self.canvas.create_text(x, y, text=text)

    def draw_graph(self):
        for n in self.nodes:
            for t in n.targets:
                self.canvas.create_line(n.vec.x, n.vec.y, t.vec.x, t.vec.y)
        for n in self.nodes:
            self.draw_node(n.vec.x, n.vec.y, n.text)

    def start_draw(self):
        self.canvas.delete("all")
        if self.busy:
            root.after_cancel(self.busy)
        random_layout(self.nodes)
        self.animate()

    def animate(self):
        self.canvas.delete("all")
        for _ in range(DELAY):
            force_layout(self.nodes)
        self.draw_graph()
        self.busy = root.after(5, self.animate)


def random_layout(nodes):
    for n in nodes:
        n.vec.x = randint(NODE_R * 4, CANVAS_WIDTH - NODE_R * 4 - 1)
        n.vec.y = randint(NODE_R * 4, CANVAS_HEIGHT - NODE_R * 4 - 1)


def f_spring(u, v):
    pass # TODO


def f_ball(u, v):
    pass # TODO


def force_layout(nodes):
    forces = {}
    for n in nodes:
        pass # TODO


g = Graph()
n1 = g.add("1")
n2 = g.add("2")
n3 = g.add("3")
n4 = g.add("4")
n5 = g.add("5")
n6 = g.add("6")
n7 = g.add("7")
n1.to(n2, n3, n4, n5)
n2.to(n5)
n3.to(n2, n4)
n6.to(n4, n1, n7)
n7.to(n5, n1)

root = Tk()
w = GUI(root)
w.nodes = g.nodes
root.mainloop()

Пример визуализации:

![](https://github.com/true-grue/kispython/blob/main/data/force.png?raw=1)

## 6. Учебный язык Фортик

В задачах этого раздела рассматривается реализация простого учебного языка. Это стековый язык (схожий с языками Forth и Postscript), в котором используется постфиксная форма записи выражений и аргументы функций, преимущественно, заранее заносятся на стек. Будем далее называть этот язык – Фортик.

Виртуальная машина Фортика имеет:

* Счетчик команд: pc.
* Стек: stack.
* Память кода: code.
* Пространство имен: scope (словарь вида имя-значение).

Подсказка: вызовы функций и работа с пространствами имен реализованы с помощью механизмов Питона.

Формат команды виртуальной машины состоит из 3 младших бит, кодирующих операцию и 29 старших бит, кодирующих аргумент операции.

Операциями являются:

* Занести значение на стек: push.
* Выполнить библиотечную операцию: op.
* Вызвать пользовательскую функцию или получить значение переменной по ключу из scope: call.
* Занести в scope аргумент как функцию: is.
* Занести в scope аргумент как переменную: to.
* Выйти из функции: exit.

В начале массива с байткодом содержится адрес точки входа (entry) – адрес выполнения главного кода.

**6.1.** (уровень сложности: средний)

Реализуйте дизассемблер байткода Фортика. Проверьте его на примерах этого раздела.

Вам понадобится фрагмент реализации виртуальной машины: коды основных операций и данные по библиотечным операциям (учитывайте порядок в словаре):

In [ ]:
OP_NAMES = {0: 'push', 1: 'op', 2: 'call', 3: 'is', 4: 'to', 5: 'exit'}

def not_implemented(vm):
    raise RuntimeError('Not implemented!')

LIB = { # Для быстрого задания большинства операций полезен модуль operator
    '+': not_implemented,
    '-': not_implemented,
    '*': not_implemented,
    '/': not_implemented, # Целочисленный вариант деления
    '%': not_implemented,
    '&': not_implemented,
    '|': not_implemented,
    '^': not_implemented,
    '<': not_implemented,
    '>': not_implemented,
    '=': not_implemented,
    '<<': not_implemented,
    '>>': not_implemented,
    'if': not_implemented,
    'for': not_implemented,
    '.': not_implemented,
    'emit': not_implemented,
    '?': not_implemented,
    'array': not_implemented,
    '@': not_implemented,
    '!': not_implemented
}

Пример использования дизассемблера:

```
>>> disasm([0, 16, 16, 1, 121, 5])
entry:
  push 2
  push 2
  op '+'
  op '.'
  exit 0
```

**6.2.** (уровень сложности: средний)

Реализуйте виртуальную машину для исполнения простейших команд байткода Фортика: сложения чисел и вывода результата на экран.
Вывод на экран в Фортике реализуется точкой. Реализованные библиотечные операции необходимо добавить в LIB.

Исходный код на языке Фортик выглядит так (этот код приведен только как иллюстрация, разбирать его не требуется до задачи на создание интерпретатора Фортика):

```Forth
2 2 + .
```

А это результат компиляции в байткод, на котором необходимо протестировать реализованную виртуальную машину.

```Python
[0, 16, 16, 1, 121, 5]
```

Заготовка виртуальной машины приведена далее:

In [ ]:
class VM:
    def __init__(self, code):
        self.stack = []
        # self.code = ...

    def run(self):
        # цикл
            # Декодирование команды, работа с pc
            if op == 'push':
                pass # TODO
            elif op == 'op':
                pass # TODO
            elif op == 'exit':
                break


**6.3.** (уровень сложности: средний)

Добавьте в виртуальную машину поддержку создания (is) и вызова (call) пользовательских функций. Также поддержите библиотечную операцию вывода символа по коду на экран (emit).

Операция call извлекает адрес вызова из scope, поэтому необходимо различать варианты хранения в scope адреса функции и значения переменной. Для этого можно хранить по ключу в scope кортеж с информацией об извлекаемом значении.

Проверьте результат на следующем байткоде:

```Python
[57, 8440, 129, 8704, 129, 8688, 129, 8600, 129, 8704, 129, 8576, 129, 8672,
 129, 8672, 129, 8576, 129, 256, 129, 8728, 129, 8712, 129, 8696, 129, 8616,
 129, 8768, 129, 8680, 129, 8688, 129, 256, 129, 8592, 129, 8792, 129, 8696,
 129, 8688, 129, 8664, 129, 8680, 129, 8616, 129, 8680, 129, 8576, 129, 264,
 129, 5, 0, 3, 2, 5]
 ```

**6.4.** (уровень сложности: высокий)

Поддержите в виртуальной машине пространство имен scope и работу с локальными переменными. Значение для локальной переменной снимается с вершины стека и записывается в scope с помощью to. Во внутренних функциях видны имена переменных из объемлющих функций, но при редактировании, как и в случае локальных переменных Питона без использования nonlocal, запись осуществляется только в переменные выполняющейся функции.

Проверьте результат на следующем байткоде:

```Python
[31, 256, 129, 5, 8, 4, 16, 12, 24, 20, 2, 121, 26, 10, 121, 26, 18, 121, 26, 5,
 32, 4, 40, 12, 34, 2, 121, 26, 10, 121, 26, 5, 0, 27, 48, 4, 24, 35, 152, 43,
 42, 2, 121, 26, 5]
```

Корректный вывод:

```Python
1 2 3 4 5 6
```

**6.5.** (уровень сложности: средний)

Добавьте в виртуальную машину поддержку ветвлений с помощью библиотечной операции if.

Работа if выглядит так:

```
значение_для_проверки адрес_функции_если_истина адрес_функции_если_ложь if
```

В исходном коде Фортика функции сами по себе безымянны и оформляются с помощью квадратных скобок. Ниже показан пример вычисления факториала:

```
[ to n  n 2 < [ 1 ] [ n n 1 - fact * ] if ] is fact
5 fact .
```

Байткод этой программы:

```Python
[17, 8, 5, 2, 2, 8, 9, 10, 17, 5, 4, 2, 16, 65, 0, 16, 105, 5, 72, 11, 40, 10,
 121, 5]
```

**6.6.** (уровень сложности: высокий)

Добавьте поддержку циклов и других недостающих операций для запуска байткода, показанного ниже.

В исходном коде Фортика цикл выглядит так:

```
кол_во_итераций повторяемая_функция for
```

При этом на входе в повторяемую функцию на стек помещается число выполненных итераций.

Вот пример вывода 10 звезд с помощью цикла:

```
10 [ to _  42 emit ] for
```

Следующий байткод реализует простой фрактал. Программа требует ввода числа от пользователя с помощью библиотечной операции ?.

Необходимо реализовать следующие библиотечные функции:

* array – выделить массив заданного размера и вернуть ссылку на него. Аргументы на стеке: размер-массива.
* @ – прочесть элемент из массива по индексу. Аргументы на стеке: ссылка-на-массив индекс-элемента.
* ! – записать элемент в массив по индексу. Аргументы на стеке: элемент ссылка-на-массив индекс-элемента.

Попробуйте ввести 495, 149 или 186.

```Python
[116, 4, 2, 2, 5, 4, 5, 80, 129, 5, 10, 18, 17, 5, 28, 280, 34, 26, 161, 5, 5,
 256, 34, 42, 50, 17, 58, 1, 161, 5, 60, 58, 66, 25, 18, 33, 76, 42, 66, 25, 18,
 33, 84, 82, 18, 17, 74, 1, 92, 98, 90, 97, 8, 41, 152, 160, 105, 5, 44, 50,
 232, 113, 5, 10, 106, 68, 50, 456, 113, 18, 25, 5, 60, 34, 42, 50, 17, 58, 1,
 153, 129, 5, 44, 50, 568, 113, 114, 5, 124, 20, 100, 8, 122, 72, 113, 52, 50,
 106, 17, 145, 36, 50, 106, 17, 104, 113, 50, 18, 25, 122, 496, 113, 10, 50,
 648, 113, 5, 0, 107, 32, 11, 48, 115, 696, 131, 137, 24, 24, 130, 5]
```

**6.7.** (уровень сложности: высокий)

Реализуйте синтаксический разбор исходного кода Фортика с получением промежуточного представления. Это представление будет далее подаваться на вход интерпретатору Фортика. Обратите внимание, что операция exit не потребуется для интерпретатора.

Например, результат разбора кода

```
[ to n  n 2 < [ 1 ] [ n n 1 - fact * ] if ] is fact
5 fact .
```

должен выглядеть так:

```Python
>>> parse(source)
[('push',
  [('to', 'n'),
   ('call', 'n'),
   ('push', 2),
   ('op', '<'),
   ('push', [('push', 1)]),
   ('push',
    [('call', 'n'),
     ('call', 'n'),
     ('push', 1),
     ('op', '-'),
     ('call', 'fact'),
     ('op', '*')]),
   ('op', 'if')]),
 ('is', 'fact'),
 ('push', 5),
 ('call', 'fact'),
 ('op', '.')]
```

Для лексического разбора достаточно воспользоваться методом split. Код хорошего варианта функции parse занимает не более 17-20 строк!

**6.8.** (уровень сложности: высокий)

Реализуйте интерпретатор для языка Фортик. Тщательно проверьте его на различных примерах, использующих все библиотечные операции.

В частности, код фрактала должен корректно выполняться:

```
[ to a  a a ] is dup
[ to a ] is drop
[ 10 emit ] is cr
[
  to gen to scale to bits
  1 gen [ drop scale * ] for to width
  width dup * array to buf
  width dup * [ to i  35 buf i ! ] for
  width scale /
  gen [ drop
    dup to block
    width [ to y
      width [ to x
        x block / scale % to loc-x
        y block / scale % to loc-y
        loc-y scale * loc-x + to loc-pos
        bits loc-pos >> 1 & [ ] [
          32 buf y width * x + !
        ] if
      ] for
    ] for
    scale /
  ] for drop
  width [ to y
    width [ to x  buf y width * x + @ emit ] for cr
  ] for
] is fractal ? 3 3 fractal
```

А также должен корректно выполняться генератор имен звезд из Elite:

```
[ to a a a ] is dup
[ to a ] is drop
[ to a 1 a [ ] if ] is exec
[ 10 emit ] is cr
1 array to seed 1 seed 0 !
[ seed 0 @ to x
  x 13 << x ^ to x
  x 17 >> x ^ to x
  x 5 << x ^ 1 32 << 1 - &
  dup seed 0 ! ] is rnd
32 array to pairs [ ] pairs 0 ! [ 108 emit 101 emit ] pairs 1 !
[ 120 emit 101 emit ] pairs 2 ! [ 103 emit 101 emit ] pairs 3 !
[ 122 emit 97 emit ] pairs 4 ! [ 99 emit 101 emit ] pairs 5 !
[ 98 emit 105 emit ] pairs 6 ! [ 115 emit 111 emit ] pairs 7 !
[ 117 emit 115 emit ] pairs 8 ! [ 101 emit 115 emit ] pairs 9 !
[ 97 emit 114 emit ] pairs 10 ! [ 109 emit 97 emit ] pairs 11 !
[ 105 emit 110 emit ] pairs 12 ! [ 100 emit 105 emit ] pairs 13 !
[ 114 emit 101 emit ] pairs 14 ! [ 97 emit ] pairs 15 !
[ 101 emit 114 emit ] pairs 16 ! [ 97 emit 116 emit ] pairs 17 !
[ 101 emit 110 emit ] pairs 18 ! [ 98 emit 101 emit ] pairs 19 !
[ 114 emit 97 emit ] pairs 20 ! [ 108 emit 97 emit ] pairs 21 !
[ 118 emit 101 emit ] pairs 22 ! [ 116 emit 105 emit ] pairs 23 !
[ 101 emit 100 emit ] pairs 24 ! [ 111 emit 114 emit ] pairs 25 !
[ 113 emit 117 emit ] pairs 26 ! [ 97 emit 110 emit ] pairs 27 !
[ 116 emit 101 emit ] pairs 28 ! [ 105 emit 115 emit ] pairs 29 !
[ 114 emit 105 emit ] pairs 30 ! [ 111 emit 110 emit ] pairs 31 !
[ 3 rnd 1 & + [ drop pairs rnd 31 & @ exec ] for ] is gen-star
100 [ drop gen-star cr ] for
```

**6.9.** (уровень сложности: высокий)

Реализуйте диалоговый режим (REPL) для интерпретатора Фортика. В процессе диалога пользовательские данные должны сохраняться.

Пример сеанса работы с Фортиком показан ниже:

```
> 2 2 + .
4
> [ to n  n n ] is dup

> [ dup * ] is square

> 5 square .
25
> 10 [ to _  42 emit ] for
**********
> [ to n  n 2 < [ 1 ] [ n n 1 - fact * ] if ] is fact

> 5 fact .
120
```

**6.10.** (уровень сложности: хакер)

Реализуйте компилятор исходного кода Фортика в байткод.

## 7. Генератор API-документации

В задачах этого раздела необходимо использовать модуль inspect.

**7.1.** (уровень сложности: высокий)

Реализуйте генератор документации на модуль Питона, задаваемый именем файла. Результат необходимо генерировать в формате Markdown.

Пример исходного модуля:

```Python
'''
Описание M.
'''


class A:
    '''
    Описание A.
    '''
    def meth1(self, x: int) -> int:
        '''
        Описание meth1.
        '''
        return x


class B:
    '''
    Описание B.
    '''
    def meth2(self):
        '''
        Описание meth2.
        '''
        pass

    def meth3(self):
        '''
        Описание meth3.
        '''
        pass


def func(a, b):
    '''
    Описание func.
    '''
    pass
```

Результат генерации:

```Markdown
# Модуль m

Описание M.

## Класс A

Описание A.

* **Метод** `meth1(self, x: int) -> int`

Описание meth1.

## Класс B

Описание B.

* **Метод** `meth2(self)`

Описание meth2.

* **Метод** `meth3(self)`

Описание meth3.

## Функция func

Сигнатура: `func(a, b)`

Описание func.
```

**7.2.** (уровень сложности: высокий)

Реализуйте инструмент визуализации иерархии модулей в проекте. Аргументом задается путь к проекту, а анализ модулей должен производиться включая и подкаталоги.

## 8. Визуализатор графики из старых компьютерных игр

Создайте визуализатор AGI-графики из старых компьютерных игр компании Sierra.

В старых играх от Sierra (например, в [King's Quest](https://www.mobygames.com/game/kings-quest) 1984 года) фоновая графика была представлена в виде последовательности [команд](https://wiki.scummvm.org/index.php?title=AGI/Specifications/Pic). В целом, результат очень напоминал векторную графику. В оригинале использовалось разрешение 160x200 пикселей, но можно попробовать перерисовать картинки из King's Quest в высоком разрешении.

**8.1.** (уровень сложности: высокий)

1. Реализуйте разбор команд из граф. файлов в каталоге `data/pic.*`.
2. Нарисуйте средствами `tkinter` результат в высоком разрешении без заливки экрана. Учитывайте, что игре используется 2 типа экранов: обычный и экран приоритетов.

Экран приоритетов использовался в играх для определения глубины сцены: таким образом можно было рисовать персонажей частично или полностью скрытыми объектами переднего плана. Экран приоритетов в наших задачах не применяется.

Код-заготовка:

In [ ]:
from pathlib import Path
import tkinter as tk

SCALE_X = 6
SCALE_Y = 4

COLORS = [
    (0, 0, 0),
    (0, 0, 168),
    (0, 168, 0),
    (0, 168, 168),
    (168, 0, 0),
    (168, 0, 168),
    (168, 84, 0),
    (168, 168, 168),
    (84, 84, 84),
    (84, 84, 252),
    (84, 252, 84),
    (84, 252, 252),
    (252, 84, 84),
    (252, 84, 252),
    (252, 252, 84),
    (252, 252, 252)
]


def draw_line(coords, color_index):
    canvas.create_line(*[(x * SCALE_X, y * SCALE_Y) for x, y in coords],
                       fill='#%02x%02x%02x' % COLORS[color_index], width=4)


def draw(pic):
    pass  # TODO


pic = Path('data/PIC.1').read_bytes()
canvas = tk.Canvas(width=160 * SCALE_X, height=170 * SCALE_Y)
canvas.pack()
draw(pic)
tk.mainloop()

Пример результата работы программы:

![](https://github.com/true-grue/kispython/blob/main/data/pic1.png?raw=1)

**8.2.** (уровень сложности: ученый)

Придумайте и реализуйте способ добавить корректную цветовую заливку областей экрана для предыдущей задачи.

## 9. Моделирование активности NPС из гипотетической ролевой игры

Используйте наследование и полиморфизм для моделирования действий NPC. Создайте лог в духе примера ниже:

```
К партии присоединяется маг Элрин23.
Разбойница Шайра17 крадётся в тени.
Орк Грогнак99 получает удар от паладина Лианор10.
Лианор10 восклицает: "Свет ведёт нас!"
Элрин23 читает заклинание: Оковы Льда.
Гоблин Мурк5 замерзает на месте.
Шайра17 наносит удар в спину: -12 HP Мурк5.
Грогнак99 рычит: "Вы заплатите за это!"
Воин Торбранд77 бросает топор: критический удар!
Скелет Воин8 разваливается на кости.
К партии присоединяется друид Элиса31.
Элиса31 призывает волка-духа.
Волк-дух кусает зомби Рэтч6: -9 HP.
Зомби Рэтч6 падает замертво.
Торбранд77 подбирает предмет: Щит Упорства.
Шайра17 обезвреживает ловушку.
Элрин23 говорит: "Нам нужно отдохнуть."
Маг Алистер8 входит в подземелье.
Орк Грашак23 атакует: человек Дориан5.
Дориан5 теряет 12 здоровья.
Эльф Тиандра17 кастует: Огненный шар.
Грашак23 получает 24 урона.
Дварф Торин91 кричит: "За сокровища предков!"
Тиандра17 находит: Свиток телепортации.
Скелет Костяшка44 разрушается.
Алистер8 получает: 120 опыта.
Торин91 ломает: Ржавый замок.
Вампир Мордред66 обращается в летучую мышь.
Дориан5 использует: Зелье лечения.
Дориан5 восстанавливает 15 здоровья.

```